# Current soln

- write a function to interpret / underatand user response
- given this users response: this is type of the output
    - what is the actual value that I should use
        - for example all dates in datetime format, etc
- inputs:
    - original question
    - output data type were looking for 
    - output options
    - user response

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def interpret_response(original_question: str, output_data_type: str, output_options, user_response: str, mistaken_response=None):
    if output_options is None:
        output_options = "Can be anything"
    else:
        output_options = ', '.join(output_options)
    
    
    # TODO - Add a check for mistaken_response and ask the user to provide the correct response if the response is incorrect.
    # TODO - implement a conifidence score to check the LLM's confidence in the response. Set a threshold to ask for user confirmation
    prompt = (
        f"Given the following question and user response, please extract the relevant value.\n\n"
        f"Question: {original_question}\n"
        f"Expected data type: {output_data_type}\n"
        f"Possible options: {output_options}\n"
        f"User response: {user_response}\n\n"
        f"Please provide the extracted value (if possible options are given, make sure that the value exists in the possible options or return 'Not applicable'):"
    )
    
    response = llm.predict(prompt).strip()
    
    if output_data_type == "number":
        print(response)
        try:
            value = float(response)
            return value
        except ValueError:
            return "Invalid response: Unable to extract a valid number."
    
    elif output_data_type == "string":
        return response
    
    elif output_data_type == "list":
        for option in output_options:
            if option.lower() in response.lower():
                return option
        return "Invalid response: Expected one of the options."
    
    else:
        return "Invalid output data type specified."



In [20]:
original_question = "Please provide your pain severity on a scale from 1 to 10."
output_data_type = "number"
output_options = ["1 (mild)", "2", "3", "4", "5 (moderate)", "6", "7", "8", "9", "10 (severe)"]
user_response = "I would say my pain is around 7"

interpreted_value = interpret_response(original_question, output_data_type, output_options, user_response)
print(f"The interpreted value is: {interpreted_value}")

7
The interpreted value is: 7.0


In [15]:
original_question = "What is your last name?"
output_data_type = "string"
output_options = None
user_response = "My name is Benjamin James Bush"

interpreted_value = interpret_response(original_question, output_data_type, output_options, user_response)
print(f"The interpreted value is: {interpreted_value}")

The interpreted value is: Bush


In [19]:
original_question = "Please provide your pain severity on a scale from 1 to 10."
output_data_type = "number"
output_options = None
user_response = "My pain is sometimes around a 7, however it is mainly a 4."

interpreted_value = interpret_response(original_question, output_data_type, output_options, user_response)
print(f"The interpreted value is: {interpreted_value}")

7, 4
The interpreted value is: Invalid response: Unable to extract a valid number.


how to continue reprompting until we have a usable response

In [24]:
original_question = "Where in your back does it hurt"
output_data_type = "string"
output_options = ['lower back', 'middle back', 'upper back']
user_response = "it hurts in the upper part of the lower back."

interpreted_value = interpret_response(original_question, output_data_type, output_options, user_response)
print(f"The interpreted value is: {interpreted_value}")

The interpreted value is: Upper back


This is where we confirm our response with the user. we can use the confirmation step as context to prevent making the same mistake